In [6]:
from typing import Dict, List
from generator import GeneratorEngine
from generator import GeneratorFactory
from prompt_settings import AdvancedCfgNetPromptSettings
from dotenv import load_dotenv
from data import Dependency
from util import get_projet_description, get_most_similar_shots, load_shots
from collections import Counter
import pandas as pd
import backoff
import json
from tqdm import tqdm


def transform(row: pd.Series) -> Dependency:
    dependency = Dependency(
        project=row["project"],
        option_name=row["option_name"],
        option_value=row["option_value"],
        option_type=row["option_type"].split(".")[-1],
        option_file=row["option_file"],
        option_technology=row["option_technology"],
        dependent_option_name=row["dependent_option_name"],
        dependent_option_value=row["dependent_option_value"],
        dependent_option_type=row["dependent_option_type"].split(".")[-1],
        dependent_option_file=row["dependent_option_file"],
        dependent_option_technology=row["dependent_option_technology"]
    )

    return dependency


@backoff.on_exception(backoff.expo, Exception, max_tries=3)
def generate(generator: GeneratorEngine, messages: List) -> str:
    response = generator.generate(messages=messages)

    if not response:
        raise Exception("Response is empty.")
    
    try:
        response_dict = json.loads(response, strict=False)
        if "isDependency" not in response_dict:
            raise Exception("KeyError: isDependency")
    except json.JSONDecodeError:
        raise Exception("Response format not serializable.")

    return response


def run_generation(df: pd.DataFrame, model_name: str) -> List[str]:

    prompt_settings = AdvancedCfgNetPromptSettings
    generator = GeneratorFactory().get_generator(
        model_name=model_name, 
        temperature=0.0
    )
    shots = load_shots()

    results = []
    project_info = []
    shot_info = []



    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        dependency = transform(row=row)

        project_str = get_projet_description(project_name=row["project"])
        context_str = row["context_str"]
        task_str = prompt_settings.get_task_str(dependency=dependency)
        shots_str = "\n\n".join([shot for shot in get_most_similar_shots(shots, dependency)])
        format_str = prompt_settings.get_format_prompt()

        system_prompt = prompt_settings.get_system_str(
            dependency=dependency,
            project_str=project_str
        )

        user_prompt = prompt_settings.advanced_query_prompt.format(
                context_str=context_str, 
                shot_str=shots_str,
                task_str=task_str,
                format_str=format_str
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        try:
            response = generate(
                generator=generator,
                messages=messages
            )
        except Exception:
            response = "None"

        shot_info.append(shots_str)
        project_info.append(project_str)
        results.append(response)

    df["responses"] = results
    df["shot_info"] = shot_info
    df["project_info"] = project_info

    df.to_csv(f"../data/analysis/failures_{model_name}.csv", index=False)

In [8]:
model_name = "llama3:70b"
df_failures = pd.read_csv("../data/analysis/failures_annotated.csv")
df_failures = df_failures[df_failures["config"] == "config2"]
df_failures= df_failures[df_failures["llm"] == model_name]

print(f"Num Failures for {model_name}: {len(df_failures)}")

env_file = "../.env"
load_dotenv(dotenv_path=env_file)
run_generation(df=df_failures, model_name=model_name)

Num Failures for llama3:70b: 152


[08/27/24 05:51:06] INFO     Ollama (llama3:70b) generator initialized.                            ]8;id=868094;file:///Users/sebastiansimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=735648;file:///Users/sebastiansimon/GitHub/cval/src/generator.py#125\125]8;;\

Processing rows:   0%|          | 0/152 [00:00<?, ?it/s]

[08/27/24 05:51:52] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=208086;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=674533;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\

Processing rows:   1%|          | 1/152 [00:45<1:55:20, 45.83s/it]

...

Processing rows:  99%|█████████▊| 150/152 [1:54:14<01:38, 49.30s/it]

[08/27/24 07:46:03] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=226756;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=944863;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\

Processing rows:  99%|█████████▉| 151/152 [1:54:56<00:47, 47.28s/it]

[08/27/24 07:46:50] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=341848;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=672146;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1038\1038]8;;\

Processing rows: 100%|██████████| 152/152 [1:55:43<00:00, 45.68s/it]
